In [1]:
!ls


'drop database.ipynb'		 'user data varidation with pydantic.ipynb'
'expelimentiong with jwt.ipynb'  'users and analyze features.ipynb'
 playlists.ipynb		  video_test.ipynb
 requirements.dev.txt


In [2]:
%cd ~/dev/video_membership/

/home/lewis/dev/video_membership


/home/lewis/dev/video_membership/env/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/lewis/dev/video_membership/env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!ls


app		env  pytest.ini        tests
connect_bundle	nbs  requirements.txt  video_membership.code-workspace


In [4]:
from app import db
from app.videos.models import Video
from cassandra.cqlengine.management import sync_table
from app.users.models import User
from app.playlists.models import Playlist
from app import config
import datetime

In [5]:
session = db.get_session()


In [6]:
settings = config.get_settings()
allow_table_drop = True

In [7]:
if allow_table_drop:
    session.execute(f"DROP TABLE IF EXISTS  {settings.keyspace}.Playlist")

In [8]:
sync_table(Video)
sync_table(Playlist)

In [9]:
obj = Playlist.objects.create(title = "hello world")
obj

Playlist(db_id=UUID('777de2b4-99d4-11ef-a70c-fcf8aedaa025'), user_id=None, updated=datetime.datetime(2024, 11, 3, 11, 11, 54, 535605), host_ids=[], title='hello world')

In [10]:
obj.updated = datetime.datetime.utcnow()
obj.save()

Playlist(db_id=UUID('777de2b4-99d4-11ef-a70c-fcf8aedaa025'), user_id=None, updated=datetime.datetime(2024, 11, 3, 11, 12, 9, 20332), host_ids=[], title='hello world')

In [11]:
obj.host_ids = ['some new lsit']
obj.save()

Playlist(db_id=UUID('777de2b4-99d4-11ef-a70c-fcf8aedaa025'), user_id=None, updated=datetime.datetime(2024, 11, 3, 11, 12, 9, 20332), host_ids=['some new lsit'], title='hello world')

In [12]:
host_ids = list(Video.objects.all().limit(5).values_list("host_id",flat=True))

In [13]:
obj.add_host_ids(host_ids=host_ids)

True

In [14]:
video_q = list(Video.objects.filter(host_id__in=obj.host_ids))

In [15]:
video_q

[title:None,urlhttps://www.youtube.com/watch?v=Ap8jTCMK3c4,
 title:None,urlhttps://www.youtube.com/watch?v=H1XxUZAOT34,
 title:None,urlhttps://www.youtube.com/watch?v=Nllgz51TcLw,
 title:None,urlhttps://www.youtube.com/watch?v=hRi_Xrs73yw,
 title:None,urlhttps://www.youtube.com/watch?v=pabciQ-MJD0]

In [16]:
print(obj.get_videos())

[title:None,urlhttps://www.youtube.com/watch?v=hRi_Xrs73yw]
